#### Installation of the necessary dependecies

In [1]:
# %pip install pyautogen
%pip install pyautogen[groq]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.0/375.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.2/345.2 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00


In [2]:
import os
from typing_extensions import Annotated
import dotenv
import pprint
from autogen.agentchat import ConversableAgent, UserProxyAgent, register_function, initiate_chat, GroupChat, GroupChatManager


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


#### Setting Up LLM

In [3]:
# For Collab

from google.colab import userdata
togeather_api_key = userdata.get('TogeatherAI_API_KEY')
Groq_api_key = userdata.get('GROQ_API_KEY')


In [ ]:
## For local jupyter
# api_key = os.environ.get('TogeatherAI_API_KEY')


In [ ]:
# # Configuration for the LLM using togeather.ai

# config_list = [
#     {
#         # 'model': 'Qwen/Qwen2-72B-Instruct',
#         "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",
#         'api_key': togeather_api_key,
#         'base_url': 'https://api.together.xyz/v1',
#         'api_type': 'together',

#         'price': [0.01, 0.02]  # Adding pricing info to suppress the warning
#     },
# ]


In [ ]:
# import os

# config_list = [
#     {
#         # Let's choose the Mixtral 8x7B model
#         "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",
#         # Provide your Together.AI API key here or put it into the TOGETHER_API_KEY environment variable.
#         "api_key": userdata.get('TogeatherAI_API_KEY'),
#         # We specify the API Type as 'together' so it uses the Together.AI client class
#         "api_type": "together",
#         "stream": False,
#     }
# ]

In [4]:
# # Configuration for the LLM using GROQ API
import os

config_list = [
    {
        # Let's choose the llama3.1 70b model or llama-3.1-70b-versatile
        "model": "llama-3.1-8b-instant",
        # "model": "llama-3.1-70b-versatile",
        # "model": "gemma2-9b-it",
        # Provide your Groq API key here or put it into the Groq_API_KEY environment variable.
        "api_key": userdata.get('GROQ_API_KEY'),
        # We specify the API Type as 'together' so it uses the Together.AI client class
        "api_type": "groq",
    }
]

### Plan for implementing Agents


**Step 1:** Reading the JSON File `(userForm.json)`
The first task is to create an agent to read and extract data from the userForm.json. This file contains important demographic, financial, and investment-related data. Based on the structure of the JSON file, We can extract:

- Demographics: Age, gender, marital status, occupation, industry.
- Financial Information: Income, investments, debt, insurance, tax category.
- Risk Tolerance: Initial investment, target value, and time interval.
- Investment Preferences: Financial goals, portfolio structure, strategy.

**Step 2:** Reanalysis by a Group of Agents
Each agent will handle a specific aspect of the user's data:

- Agent 1: Analyze financial goals (investment goals, time duration).
- Agent 2: Reevaluate the current financial situation (income, investments, debt, etc.).
- Agent 3: Assess risk tolerance based on provided financial and investment preferences.
- Agent 4: Review the portfolio structure and investment strategy.
Each agent will return an updated analysis and risk tolerance adjustment.

**Step 3:** Writing the New JSON File
Create a final agent that gathers the outputs from the agents in Step 2 and compiles them into a new JSON file. This file will include the revised risk tolerance values based on the analysis, with updated parameters reflecting the user’s current financial situation and risk preferences.



#### Step 1: Read and Extract Data from the JSON File
The first agent will extract all the necessary data from the `userForm.json` file, such as demographic, financial, and investment-related information.

**Agent:** `DataReader`
This agent is responsible for reading the JSON file and making the data available for further analysis.

In [31]:
import json
from pathlib import Path

from typing import Union


# Directory where the userForm.json is stored
json_file_path =  "/content/userForm.json"

# Function to read the JSON file
def read_json_data(file_path: str) -> Union[dict, str]:
    """
    Reads and loads JSON data from the specified file path.

    Parameters:
    - file_path (str): The path to the JSON file.

    Returns:
    - dict: The JSON data as a dictionary if the file is successfully read.
    - str: An error message if the file is not found.
    """
    json_file_path =  "/content/userForm.json"
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        return "Error: JSON file not found."


# Function to be registered, reading from the JSON file
def get_json_data(query: str) -> str:
    """
    Retrieves JSON data from the file and returns it as a pretty-printed JSON string.

    Parameters:
    - query (str): A query string (not used in this function but kept for interface consistency).

    Returns:
    - str: The JSON data as a pretty-printed string, or an error message if the file is not found.
    """
    data = read_json_data(json_file_path)
    return json.dumps(data, indent=4)  # Return a pretty-printed JSON string for clarity


##########      AGENT         #########

# LLM-powered agent that will query JSONTool
data_reader_agent = ConversableAgent(
    name="DataReader",
    system_message="You are an AI assistant responsible for extracting specific information from a JSON file."
                   "You will ask JSONTool to read the file and extract the following data:"
                   "Demographics: Age, gender, marital status, occupation, industry."
                   "Financial Information: Income, investments, debt, insurance, tax category,"
                   "financials, risk tolerance, and investment preferences."
                   "If the json_reader_tool_agent successfully loaded the json file, then give the answer of the question asked from you, and then after the satisfactory answer return the 'TERMINATE' message to end the conversation."
                   "Return 'TERMINATE' to end the conversation.",
    llm_config={"config_list": config_list},  # Set your model configuration here
    # is_termination_msg=lambda x: x.get("content", "") and (x.get("content", "").rstrip().endswith("TERMINATE") or x.get("content", "") == ""),
    human_input_mode="NEVER",
    # max_consecutive_auto_reply=10,
)


# Tool agent with JSON reading capabilities
json_reader_tool_agent = UserProxyAgent(
    name="JSONTool",
    # llm_config={"config_list": config_list},  # No LLM, it's just a tool to read the file
    system_message="You are a tool that reads JSON file "
                   "If you have the information you need, simply reply with 'TERMINATE'.",
    # default_auto_reply="JSON reading complete. Return 'TERMINATE' to finish.",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and (x.get("content", "").rstrip().endswith("TERMINATE") or x.get("content", "") == ""),
    # max_consecutive_auto_reply=10,
)


# Register the JSON reading function with the JSONTool agent
register_function(
    get_json_data,
    name="get_json_data",
    caller = data_reader_agent,
    executor=json_reader_tool_agent,
    description="Read data from the JSON file and return it.",
)


In [58]:

# query = "Please give something about the demographics, financial information, risk tolerance, and investment preferences."
# # Initiate chat where DataReader asks JSONTool to read the file
# result = json_reader_tool_agent.initiate_chat(
#     recipient=data_reader_agent,
#     message=query,
#     summary_method="reflection_with_llm",
#     summary_args={
#     "summary_prompt": """Summarize the conversation"""
# }
# )


### Step 2: Reanalysis by a Group of Agents
We will now create four agents that will perform specialized tasks based on the extracted data. Each agent will analyze specific sections and return an updated risk tolerance.

#### Agent Setup
Each agent will have specific functionality:

- `FinancialGoalsAgent:` Analyzes investment goals and time duration.
- `FinancialSituationAgent:` Reevaluates income, investments, debt, etc.
- `RiskToleranceAgent:` Assesses the user’s risk tolerance.
- `PortfolioStructureAgent:` Reviews portfolio structure and investment strategy.

In [32]:
# --------------- Agent 1: Financial Goals Agent ---------------
financial_goals_agent = ConversableAgent(
    name="FinancialGoalsAgent",
    system_message="""You are an expert in analyzing financial goals. Your task is to analyze the user's investment goals and the time duration for achieving them. Review the given JSON data and provide a comprehensive analysis.""",
    llm_config={"config_list": config_list},  # Set your model configuration here
)

# --------------- Agent 2: Financial Situation Agent ---------------
financial_situation_agent = ConversableAgent(
    name="FinancialSituationAgent",
    system_message="""You are an expert in reevaluating a user's financial situation, including income, investments, and debt. Based on the JSON data provided, review and assess the user's financial situation.""",
    llm_config={"config_list": config_list},  # Set your model configuration here
)

# --------------- Agent 3: Risk Tolerance Agent ---------------
risk_tolerance_agent = ConversableAgent(
    name="RiskToleranceAgent",
    system_message="""You specialize in assessing risk tolerance. Your task is to analyze the user's risk tolerance based on the financial data, investment preferences, and any other relevant details in the JSON data.""",
    llm_config={"config_list": config_list},  # Set your model configuration here
)

# --------------- Agent 4: Portfolio Structure Agent ---------------
portfolio_structure_agent = ConversableAgent(
    name="PortfolioStructureAgent",
    system_message="""You are an expert in reviewing portfolio structure and investment strategies. Your job is to review the user's current portfolio structure based on the JSON data and suggest any changes necessary to align with the user's goals and risk tolerance.""",
    llm_config={"config_list": config_list},  # Set your model configuration here
)

### Step 3: Writing the New JSON File
After the agents complete their analyses, we create a final agent that collects the findings and writes them into a new JSON file with the updated risk tolerance values.

`Agent:` `WriterAgent`
This agent compiles all the inputs from the previous agents and updates the JSON file with the new risk tolerance values.

- `WriterAgent:` Compiles the results from the other agents and returns the data as a JSON-formatted string.

- `ExecutorAgent:` Executes the function convert_string_to_json_file to save the JSON string as a file, using the string output from WriterAgent.

- `egister the Function:` We'll register the `convert_string_to_json_file` function with the ExecutorAgent so it can be called during execution.

In [33]:
# --------------- Step 1: Define WriterAgent ---------------
writer_agent = ConversableAgent(
    name="WriterAgent",
    system_message="""You are a data-writing assistant responsible for compiling analysis results into a JSON format.
    Always respond strictly in valid JSON format. Here's an example format of your response:
    {
        "financial_goals_analysis": {
            "goal_status": "on track",
            "time_duration": "5 years"
        },
        "financial_situation_analysis": {
            "income": "stable",
            "debt": "manageable",
            "investment": "sufficient"
        },
        "risk_tolerance_analysis": {
            "updated_risk_tolerance": "medium"
        },
        "portfolio_structure_analysis": {
            "suggestions": "diversify with bonds and equities"
        }
    }
    Your task is to merge the analysis results into this format and return valid JSON as a response.""",
    llm_config={"config_list": config_list},  # Set your model configuration here
)


# --------------- Step 2: Define ExecutorAgent ---------------
executor_agent = UserProxyAgent(
    name="ExecutorAgent",
    system_message="You are an executor responsible for executing the provided functions. You will receive JSON data as input and execute the function to save it as a file.",
    # code_execution_config={
    #     "work_dir": "./",  # Directory where the JSON file will be saved
    #     "use_docker": False  # Set to True if you want to use Docker for execution (recommended for security)
    # },
    human_input_mode="NEVER"
)


# --------------- Step 3: Define the function to convert JSON string to file ---------------


from typing import Any
# json_string =
# Output file path
output_file_path = "/content/updated_userForm.json"

def convert_string_to_json_file(json_string: str, output_file_path: str) -> str:
    """
    Converts a JSON string into a JSON file and saves it to the specified path.

    Parameters:
    - json_string (str): The JSON data as a string.
    - output_file_path (str): The path where the JSON file will be saved.

    Returns:
    - str: Success message if the file is created successfully, or an error message if JSON decoding fails.
    """
    output_file_path = "/content/updated_userForm.json"

    try:
        # Parse the string into a JSON object
        json_data = json.loads(json_string)

        # Write the JSON data to a file
        with open(output_file_path, 'w') as json_file:
            json.dump(json_data, json_file, indent=4)

        return "JSON file successfully created at " + output_file_path
    except json.JSONDecodeError as e:
        return "Failed to decode JSON: " + str(e)

# Register the function with the executor agent
register_function(
    convert_string_to_json_file,
    name="convert_string_to_json_file",
    caller=writer_agent,
    executor=executor_agent,
    description="Function that converts a JSON string to a JSON file.",
)


#### Agent Flow
- `Admin Agent:` Acts as a human proxy. This agent interacts with the planner and provides final approval for the execution plan.
- `Planner Agent:` Creates and revises the plan. It suggests which agent will perform which task (e.g., JSON reading, financial reanalysis, and JSON writing).

In [35]:
# Step 1: Define the Admin agent
admin_agent = UserProxyAgent(
    name="Admin",
    system_message="""You are the Admin. Your role is to interact with the Planner to discuss and approve the plan before execution.
    The plan execution must be approved by you before proceeding.""",
    code_execution_config=False,  # Admin will not execute code but will approve plans.
    human_input_mode="NEVER",
)

# Step 2: Define the Planner agent
planner_agent = ConversableAgent(
    name="Planner",
    system_message="""Planner. Your role is to suggest a plan for reading data from the JSON file (userForm.json), analyzing financial information, and updating the JSON file.
    The process includes 3 steps:

    Step 1: Read the JSON File (userForm.json). This file contains demographic, financial, and investment-related data.
    You will extract the following information:
    - Demographics: Age, gender, marital status, occupation, industry.
    - Financial Information: Income, investments, debt, insurance, tax category.
    - Risk Tolerance: Initial investment, target value, and time interval.
    - Investment Preferences: Financial goals, portfolio structure, strategy.

    Step 2: Reanalysis using specialized agents:
    - Agent 1: Analyze financial goals (investment goals, time duration).
    - Agent 2: Reevaluate the current financial situation (income, investments, debt, etc.).
    - Agent 3: Assess risk tolerance based on financial and investment preferences.
    - Agent 4: Review the portfolio structure and investment strategy.

    Each agent will return an updated analysis and risk tolerance adjustment.

    Step 3: Write the new JSON file with the updated risk tolerance values.

    Discuss this plan with the Admin and revise it based on feedback until approval.""",
    llm_config={"config_list": config_list},  # Set your model configuration here
)






In [53]:

# Description for each agent

# Admin Agent
admin_agent.description = "Admin responsible for overseeing the entire process, including interacting with the Planner to discuss and approve plans before execution."
# Data Reader Agent
data_reader_agent.description = "Agent tasked with reading and extracting data from the JSON file. It can extract demographics, financial information, risk tolerance, and investment preferences and all the various kind of data by calling JSON tool."
# JSON Tool Agent
json_reader_tool_agent.description = "Tool agent responsible for reading data from the JSON file. It does not have LLM capabilities but performs file reading functions."
# Financial Goals Agent
financial_goals_agent.description = "Agent specializing in analyzing financial goals. It reviews investment goals and time duration to provide a comprehensive analysis."
# Financial Situation Agent
financial_situation_agent.description = "Agent focusing on reevaluating the user's financial situation. It assesses income, investments, debt, and other financial details."
# Risk Tolerance Agent
risk_tolerance_agent.description = "Agent dedicated to assessing risk tolerance based on financial data and investment preferences. It provides an updated risk tolerance analysis."
# Portfolio Structure Agent
portfolio_structure_agent.description = "Agent responsible for reviewing the portfolio structure and investment strategy. It suggests changes to align with goals and risk tolerance."
# Writer Agent
writer_agent.description = "Agent responsible for compiling analysis results into a JSON format. It merges results from other agents into a single JSON file with updated parameters."
# Executor Agent
executor_agent.description = "Agent responsible for executing the function to save the final JSON file with updated risk tolerance values."
# Planner Agent
planner_agent.description = "Agent that suggests a plan for reading and analyzing JSON data, reanalyzing financial information, and updating the JSON file. It discusses the plan with Admin and revises it based on feedback."


# Define group chat
group_chat = GroupChat(
    messages=[],
    agents=[
        admin_agent,
        planner_agent,
        data_reader_agent,
        json_reader_tool_agent,
        financial_goals_agent,
        financial_situation_agent,
        risk_tolerance_agent,
        portfolio_structure_agent,
        writer_agent,
        executor_agent
    ],

    admin_name="Admin",
    speaker_transitions_type="allowed",
    send_introductions=True,
    allowed_or_disallowed_speaker_transitions={
    admin_agent: [planner_agent, data_reader_agent],  # Admin oversees and communicates primarily with Planner for plan approval
    planner_agent: [
        data_reader_agent,
        # json_reader_tool_agent
        # financial_goals_agent,
        # financial_situation_agent,
        # risk_tolerance_agent,
        # portfolio_structure_agent,
        # writer_agent  # Planner coordinates analysis and writing
    ],
    data_reader_agent: [json_reader_tool_agent],  # DataReader works with JSONToolAgent and updates Planner on data read progress
    json_reader_tool_agent: [data_reader_agent, financial_goals_agent],  # JSONToolAgent only communicates with DataReader for file reading
    financial_goals_agent: [data_reader_agent,financial_situation_agent],  # FinancialGoals provides analysis and works with WriterAgent to update data
    financial_situation_agent: [data_reader_agent, risk_tolerance_agent],  # FinancialSituation provides analysis and works with WriterAgent
    risk_tolerance_agent: [data_reader_agent,portfolio_structure_agent],  # RiskTolerance communicates results to WriterAgent
    portfolio_structure_agent: [data_reader_agent, planner_agent,portfolio_structure_agent, writer_agent],  # PortfolioStructure communicates with WriterAgent
    writer_agent: [data_reader_agent, portfolio_structure_agent, executor_agent],  # Writer compiles data and passes final instructions to ExecutorAgent
    executor_agent: [writer_agent],  # Executor finalizes the process by saving updated JSON
},

    max_round=20,
)

# Define group chat manager
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"config_list": config_list},  # Set your model configuration here
)

In [54]:
result = admin_agent.initiate_chat(
          group_chat_manager,
          message="""
          Reevaluate the user's risk tolerance values based on the provided JSON data, analyse relevant financial
          information,compile the findings and write into a new JSON file with updated values.
          """,
          summary_method="reflection_with_llm",
          summary_args={
              "summary_prompt": """Summarize the conversation"""
          }
)

Admin (to chat_manager):


          Reevaluate the user's risk tolerance values based on the provided JSON data, analyse relevant financial
          information,compile the findings and write into a new JSON file with updated values.
          

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

**Plan for Reevaluating User's Risk Tolerance**

**Step 1: Read JSON File and Extract Relevant Information**

1. Use the `JSONTool` to read the `userForm.json` file.
2. Pass the extracted JSON data to the `DataReader` to extract the following information:
	* Demographics: Age, gender, marital status, occupation, industry.
	* Financial Information: Income, investments, debt, insurance, tax category.
	* Risk Tolerance: Initial investment, target value, and time interval.
	* Investment Preferences: Financial goals, portfolio structure, strategy.

**Step 2: Reanalyze Financial Informati

In [57]:
pprint.pprint(result.summary['content'])

('The JSON file contains the updated user information based on the analysis of '
 'the provided JSON data.\n'
 '\n'
 '**Updated User Information:**\n'
 '\n'
 '`{\n'
 "  'risk_tolerance': 'Moderate to High',\n"
 "  'financial_info': {\n"
 "    'income': 1000000,\n"
 "    'investments': ['Real estate'],\n"
 "    'debt': 100000,\n"
 "    'insurance_coverage': 50000,\n"
 "    'tax_category': '5-15 lakhs'\n"
 '  },\n'
 "  'retirement_plans': {\n"
 "    'goals': ['Buy a new house at age 35'],\n"
 "    'lifestyle': ['Buy a new house']\n"
 '  },\n'
 "  'financial_goals': {\n"
 "    'short_term_goals': ['Buy a new house within 10 years'],\n"
 "    'long_term_goals': ['Achieve a balance between long-term returns and "
 "short-term losses']\n"
 '  },\n'
 "  'recommendations': {\n"
 "    'reduce_debt': True,\n"
 "    'increase_savings_rate': True,\n"
 "    'diversify_investments': True,\n"
 "    'prioritize_long_term_security': True,\n"
 "    'consult_financial_advisor': True\n"
 '  }\n'
 '}`\n'
 